<h2>C - Network

In [2]:
import pandas as pd
import icecream as ic
import numpy as np
import plotly
import matplotlib as plt
import xml.etree.ElementTree as ET
import pandas as pd
import psycopg2
import psycopg2.extras
import csv
import json
from tqdm import tqdm

C1: small-molecule roles

In [61]:
#chebi database

hostname = 'localhost'
database = 'Chebi'
username = 'postgres'
pwd = 'wlf1115'
port_id = 5432
conn = None

query = 'Acetazolamide'
with psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id) as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:

        query_script = '''SELECT type, final_id
                        FROM relation
                        WHERE init_id = (
                            SELECT compound_id
                            FROM names
                            WHERE name IN (%s)
                            )'''
        cur.execute(query_script, (query,))
        
        type_id = {}
        type_compound = {}
        for record in cur.fetchall():
            type_id[record[0]] = []
            type_compound[record[0]] = []
            type_id[record[0]] = record[1]


        for k,v in type_id.items():
            # query1 = '''SELECT name
            #             FROM names
            #             WHERE compound_id IN ({}
            #             )'''.format(tuple([x]))
            query_script1 = '''SELECT name
                        FROM names
                        WHERE compound_id IN (%s)
                        '''
            query1 = tuple([v])
            cur.execute(query_script1, query1)
            for record in cur.fetchall():
                type_compound[k].append(record)

print(query)
type_compound    

Acetazolamide


{'has_part': [['Diamox'],
  ['Sodium acetazolamide'],
  ['sodium [(5-acetamido-1,3,4-thiadiazol-2-yl)sulfonyl]azanide'],
  ['N-(5-Sulfamoyl-1,3,4-thiadiazol-2-yl)acetamide monosodium salt']],
 'is_conjugate_base_of': [['[(5-acetamido-1,3,4-thiadiazol-2-yl)sulfonyl]azanide'],
  ['acetazolamide']]}

C2: metabolic pathways

In [ ]:
# https://www.ebi.ac.uk/biomodels/MODEL1109130000#Files #!how to get binary data, how is the graph represented # read paper
# We downloaded the reconstruction of human metabolism (Recon)57 from Pathway Commons58 (http://www.pathwaycommons.org, July 2017) in binary interaction form. 
# Data were represented as an undirected graph where nodes are metabolites and edges denote reactions. 

In [120]:
recon_df = pd.read_csv("../PathwayCommons.8.reconx.BINARY_SIF.hgnc.txt", sep='\t', names=['init', 'process','affected'])
recon_df #how to convert protein name to abbreviation

,init,process,affected
0,A4GALT,catalysis-precedes,B3GALNT1
1,A4GALT,catalysis-precedes,B3GNT3
2,A4GALT,controls-production-of,CHEBI:58223
3,A4GALT,catalysis-precedes,ENTPD4
4,A4GALT,catalysis-precedes,NME1
...,...,...,...
52598,ZADH2,controls-production-of,CHEBI:17790
52599,ZADH2,controls-production-of,CHEBI:18041
52600,ZADH2,controls-production-of,CHEBI:28972
52601,ZADH2,catalysis-precedes,GLO1


C3: signaling pathways

In [ ]:
#  C3 space (and the C4 and C5 spaces) is aimed at any molecule with known protein targets. 
# In this case, we list the biological pathways that may be affected by the interaction of a molecule with its targets. 
# Human pathways were collected from Reactome (https://reactome.org, May 2017) #! use of graphDB

# we chose to use binding activities from B4, since this is an extensive dataset containing mostly literature data with well-accepted activity thresholds. 
# In B4, 24.5% of the compound–protein interactions do not correspond to human proteins. #!need B4 data
# These were mapped to their human orthologs using MetaPhOrs60 (http://orthology.phylomedb.org, May 2017), following the observation that binding activities can be safely transferred between orthologous proteins61,  #!how is it done, it is a database?
# especially if they belong to closely related species, as it is the case for B4 data62. Of all the nonhuman proteins mapped to the human orthologs, 94.4% were mammal proteins #!http://orthology.phylomedb.org/download

In [7]:
reactome_df = pd.read_csv("../UniProt2Reactome_All_Levels.txt", sep='\t', names=['uniprot_id','interaction_code','web','pathway','IEA','organism'])
reactome_df = reactome_df[['uniprot_id','pathway']]

chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
chemble_B1_target_uniprot = chemble_B1[chemble_B1['compound_name']=='Acetazolamide']['uniprot_id']



/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_1245/1950680321.py:4: DtypeWarning: Columns (5,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)


In [18]:
#  biological pathways that may be affected by the interaction of a molecule with its targets. 
# ! still need to make sure all the non human genes are turned into human orthologs
reactome_df[reactome_df['uniprot_id'].isin(chemble_B1_target_uniprot)]


,uniprot_id,pathway
412824,O00255,Signal Transduction
412825,O00255,Signaling by TGF-beta Receptor Complex
412826,O00255,Signaling by Rho GTPases
412827,O00255,RHO GTPase Effectors
412828,O00255,Signaling by WNT
...,...,...
903808,Q9Y6L6,Disorders of transmembrane transporters
903809,Q9Y6L6,Transport of organic anions
903810,Q9Y6L6,Metabolism of steroids
903811,Q9Y6L6,Drug ADME


C4: biological processes

In [6]:
with open('../GOA/25.H_sapiens.goa', 'r') as handle:

    for rec in gafiterator(handle):
        print(rec)
        break

{'DB': 'UniProtKB', 'DB_Object_ID': 'A0A024QZ33', 'DB_Object_Symbol': 'NSRP1', 'Qualifier': ['involved_in'], 'GO_ID': 'GO:0000381', 'DB:Reference': ['GO_REF:0000002'], 'Evidence': 'IEA', 'With': ['InterPro:IPR018612'], 'Aspect': 'P', 'DB_Object_Name': 'Nuclear speckle splicing regulatory protein 1', 'Synonym': ['NSRP1', 'CCDC55', 'hCG_1646942'], 'DB_Object_Type': 'protein', 'Taxon_ID': ['9606'], 'Date': '20220907', 'Assigned_By': 'InterPro', 'Annotation_Extension': '', 'Gene_Product_Form_ID': ''}


In [17]:
# We downloaded the Gene Ontology Annotation database (https://www.ebi.ac.uk/GOA, May 2017)
# read the ‘biological process’ branch of the ontology as a directed acyclic graph (‘is a’ relationships). 
# Proteins were annotated with their Gene Ontology Annotation biological process terms plus parent terms (up to the root of the directed acyclic graph). 
# Similar to C3, we associated molecules with biological process terms by simply checking the annotations of the molecule targets (B4).
from ast import And
from Bio.UniProt.GOA import gafiterator, record_has

for x in chemble_B1_target_uniprot.to_list():
    Interpro_ID = {'With': set([f'InterPro:IPR018612'])} 
    DB_Object_ID = {'DB_Object_ID': set([x])} 

    # DB_Object_name = {'DB_Object_name' : set(['Nuclear speckle splicing regulatory protein 1'])} 

    with open('../GOA/25.H_sapiens.goa', 'r') as handle:

        for rec in gafiterator(handle):
            if record_has(rec, DB_Object_ID):
                for key in ('DB_Object_ID','DB_Object_Name', 'Qualifier', 'GO_ID'):
                    print(rec[key])
                print('='*20)
        break

P00915
Carbonic anhydrase 1
['enables']
GO:0004064
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0005515
P00915
Carbonic anhydrase 1
['enables']
GO:0005515
P00915
Carbonic anhydrase 1
['enables']
GO:0008270
P00915
Carbonic anhydrase 1
['enables']
GO:0016829
P00915
Carbonic anhydrase 1
['enables']
GO:0016836
P00915
Carbonic anhydrase 1
['enables']
GO:0016836
P00915
Carbonic anhydrase 1
['enables']
GO:0018820
P00915
Carbonic anhydrase 1
['enables']
GO:0046872
P00915
Carbonic anhydrase 1
['involved_in']
GO:0006730
P00915
Carbonic anhydrase 1
['involved_in']
GO:0006730
P00915
Carbonic anhydrase 1
['is_active_in']
GO:0005737
P00915
Carbonic anhydrase 1
['located_in']
GO:0005737
P00915
Carbonic

# C5: interactomes

In [68]:
# We collected five representative PPI networks, 
# namely STRING (score of >700, that is, high confidence)63 (v.10, https://string-db.org) (14,725 proteins (p), 300,686 interactions (i)), #! waiting for data
# InWeb (score of 0.5)64 (http://www.intomics.com/inbio/map, March 2017) (10,100 p, 168,970 i), #!paid service? <- not gonna use
# a portion of Pathway Commons containing interactions from known pathways (Kyoto Encylopedia of Genes and Genomes65, NetPath66, PANTHER67 and WikiPathways68) (9,344 p, 242,962 i), #!need to find out
# an in-house network of physical binary PPIs69 (13,038 p, 64,659 i),  #!need to find in house pipeline

In [3]:
def influence_node(input_array, r_min=0, r_max=10): #maximum of the range of your measurement
    input_max = np.amax(input_array) #maximum of the range of your desired target scaling
    input_min = np.amin(input_array) #minimum of the range of your desired target scaling
    output = np.ndarray.round((input_array - input_min) / (input_max - input_min) * (r_max - r_min) + r_min)
    return output

In [4]:
drugbank_df = pd.read_csv("DB/drugbank_extracted.csv")
target_list_uniprot = drugbank_df[drugbank_df["dg_name"]=="Acetazolamide"]['target_uniprot'].tolist()
target_list = stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id'].isin(target_list_uniprot)]['protein1'].unique().tolist()

NameError: name 'stringDB_uniprot' is not defined

In [5]:
#determine weight using assay data from B4
chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
chemble_B4B5 = pd.read_csv("../chembl31forlinfeng/B4B5_data.csv", sep=',', header=0)
chemble_B4B5_ic50 = chemble_B4B5[chemble_B4B5['standard_type']=='IC50'] #B4

ion_channels = ['Ligand-gated ion channel','Voltage-gated ion channel', 'Other ion channel']
g_protein = ['Family A G protein-coupled receptor', 'Family C G protein-coupled receptor',
            'Family B G protein-coupled receptor','Frizzled family G protein-coupled receptor',
            'Taste family G protein-coupled receptor']

not_other = ['Ligand-gated ion channel','Voltage-gated ion channel', 'Other ion channel'
            'Family A G protein-coupled receptor', 'Family C G protein-coupled receptor',
            'Family B G protein-coupled receptor','Frizzled family G protein-coupled receptor',
            'Taste family G protein-coupled receptor', 'Kinase','Nuclear receptor']
# kinases ≤30 nM, 
kinase_df = chemble_B1[chemble_B1['l2']=="Kinase"]
kinase_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(kinase_df['uniprot_id'])]
kinase_ic_filter = chemble_B4B5_ic50[chemble_B4B5_ic50['standard_value']<=30]
# G protein-coupled receptors ≤100 nM, 
g_protein_df = chemble_B1[chemble_B1['l2'].isin(g_protein)]
g_protein_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(g_protein_df['uniprot_id'])]
g_protein_ic_filter = chemble_B4B5_ic50[chemble_B4B5_ic50['standard_value']<=100]
# nuclear receptors ≤100 nM,  
nuclear_receptor_df = chemble_B1[chemble_B1['l2']=='Nuclear receptor']
nuclear_receptor_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(nuclear_receptor_df['uniprot_id'])]
nuclear_receptor_ic_filter = chemble_B4B5_ic50[chemble_B4B5_ic50['standard_value']<=100]
# ion channels ≤10 μM
ion_channel_df = chemble_B1[chemble_B1['l2'].isin(ion_channels)]
ion_channel_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(ion_channel_df['uniprot_id'])]
ion_channel_ic_filter = chemble_B4B5_ic50[chemble_B4B5_ic50['standard_value']<=1000]
# others ≤1 μM. 

others_df = chemble_B1[~chemble_B1['l2'].isin(not_other)]
other_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(others_df['uniprot_id'])] #!数据重复是怎么回事
other_ic_filter = other_ic[other_ic['standard_value']<=1000]
# We also kept activities one order of magnitude lower than the class-specific cutoff (to a maximum of 10 μM), 
# and gave these annotations half the weight in downstream analyses (that is, log10 scaling)

in_threshold = pd.concat([kinase_ic_filter, g_protein_ic_filter,nuclear_receptor_ic_filter,ion_channel_ic_filter,other_ic_filter])


/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_34974/1745851512.py:2: DtypeWarning: Columns (5,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_34974/1745851512.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B4B5 = pd.read_csv("../chembl31forlinfeng/B4B5_data.csv", sep=',', header=0)


#### StringDB

In [6]:
stringDB_uniprot = pd.read_csv('../human_HighConfidence_string_PPI_add_uniprot.csv')
stringDB_uniprot.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_uniprot_id,protein1_protein_name,protein2_uniprot_id,protein2_protein_name
0,9606.ENSP00000000233,9606.ENSP00000324287,0,0,0,49,147,0,736,767,P84085,ADP-ribosylation factor 5,Q15057,"Arf-GAP with coiled-coil, ANK repeat and PH do..."
1,9606.ENSP00000000233,9606.ENSP00000387286,0,0,0,79,248,600,141,730,P84085,ADP-ribosylation factor 5,P62820,Ras-related protein Rab-1A (EC 3.6.5.2) (YPT1-...
2,9606.ENSP00000000233,9606.ENSP00000262812,0,0,0,190,163,600,173,745,P84085,ADP-ribosylation factor 5,O14579,Coatomer subunit epsilon (Epsilon-coat protein...
3,9606.ENSP00000000233,9606.ENSP00000158762,0,0,0,0,147,0,736,765,P84085,ADP-ribosylation factor 5,Q15027,"Arf-GAP with coiled-coil, ANK repeat and PH do..."
4,9606.ENSP00000000233,9606.ENSP00000449270,0,0,0,162,210,600,194,757,P84085,ADP-ribosylation factor 5,P61923,Coatomer subunit zeta-1 (Zeta-1-coat protein) ...


In [10]:
stringDB_df = pd.read_csv('../human_HighConfidence_string_PPI.csv', sep=' ')

Generating hotnet input files

In [14]:
#create input for hotnet -stringDB
bla = np.append(stringDB_df['protein1'].unique(), stringDB_df['protein2'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
bla = set(bla.tolist())

protein_encoding = {}

for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in stringDB_df['protein1']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in stringDB_df['protein2']:
    protein2_encoding.append(protein_encoding[x])
    
stringDB_df['protein1_encoding'] = protein1_encoding
stringDB_df['protein2_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('stringDB_index.txt', sep = " ", index = False, index_label = False, header = False)
stringDB_df[['protein1_encoding','protein2_encoding']].to_csv('stringDB_edge.txt', sep = " ", index = False, index_label = False, header = False)

In [7]:
bla = np.append(stringDB_uniprot['protein1_uniprot_id'].unique(), stringDB_uniprot['protein2_uniprot_id'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
bla = set(bla.tolist())

protein_encoding = {}

for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in stringDB_uniprot['protein1_uniprot_id']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in stringDB_uniprot['protein2_uniprot_id']:
    protein2_encoding.append(protein_encoding[x])
    
stringDB_uniprot['protein1_encoding'] = protein1_encoding
stringDB_uniprot['protein2_encoding'] = protein2_encoding

stringDB_uniprot = stringDB_uniprot[stringDB_uniprot['protein2_uniprot_id']!='nan']
stringDB_uniprot = stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id']!='nan']

In [14]:
arr1 = stringDB_uniprot['protein1_uniprot_id'].unique()
arr2 = stringDB_uniprot['protein2_uniprot_id'].unique()
arr3 = np.concatenate((arr1, arr2))
stringDB_uniprot_conv = stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id'].isin(arr3)]

In [19]:
#! Not used
# #create input for hotnet -stringDB
# bla = np.append(stringDB_uniprot['protein1_uniprot_id'].unique(), stringDB_uniprot['protein2_uniprot_id'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
# bla = set(bla.tolist())

# protein_encoding = {}

# for i, x in enumerate(bla):
#     protein_encoding[x] = i+1
# protein1_encoding = []
# for x in stringDB_uniprot['protein1_uniprot_id']:
#     protein1_encoding.append(protein_encoding[x])
    
# protein2_encoding = []
# for x in stringDB_uniprot['protein2_uniprot_id']:
#     protein2_encoding.append(protein_encoding[x])
    
# stringDB_uniprot['protein1_encoding'] = protein1_encoding
# stringDB_uniprot['protein2_encoding'] = protein2_encoding

# protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

# #export index file
# protein_encoding[['Encodings','Names']].to_csv('stringDB_index1.txt', sep = " ", index = False, index_label = False, header = False)
# stringDB_uniprot[['protein1_encoding','protein2_encoding']].to_csv('stringDB_edge1.txt', sep = " ", index = False, index_label = False, header = False)

Opening H5 and get the influence matrix

In [8]:
import h5py
with h5py.File('/Users/linfengwang/Github/linfeng-wang-CCdatabase/hotnet_output/stringDB_mat_ppr_0.6.h5', "r") as hdf:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % hdf.keys())
    PPR = hdf.get('PPR')
    PPR_dataset = np.array(PPR)
    
    edges = hdf.get('edges') #sample name, given in pairs
    edges_dataset = np.array(edges)

    nodes = hdf.get('nodes') #individual sample names
    nodes_dataset = np.array(nodes) 

    network_name = hdf.get('network_name') #name of the network
    network_name_dataset = np.array(network_name)
    beta = hdf.get('beta') #version beta
    beta_dataset = np.array(beta)

    # # get first object name/key; may or may NOT be a group
    # a_group_key = list(f.keys())[0]

    # # get the object type for a_group_key: usually group or dataset
    # print(type(f[a_group_key])) 

    # # If a_group_key is a group name, 
    # # this gets the object names in the group and returns as a list
    # data = list(f[a_group_key])

    # # If a_group_key is a dataset name, 
    # # this gets the dataset values and returns as a list
    # data = list(f[a_group_key])
    # # preferred methods to get dataset values:
    # ds_obj = f[a_group_key]      # returns as a h5py dataset object
    # ds_arr = f[a_group_key][()]  # returns as a numpy array

Keys: <KeysViewHDF5 ['PPR', 'beta', 'edges', 'network_name', 'nodes']>


In [9]:
target_list_stringDB = stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id'].isin(target_list_uniprot)]['protein1'].tolist()

In [16]:
target_edge = []
for target in target_list_stringDB:
    for x in edges_dataset:
        # print(x[0].decode('ascii'))
        if x[0].decode('ascii') == target:
            target_edge.append(x)
        # elif x[1].decode('ascii') == target:
        #     target_edge.append(x)

# is is only protein1 influencing protein2 right? 
            
# #convert the edge name to index
target_edge_coord = []
for x in target_edge:
    coord = ['nan', 'nan']
    for i, y in enumerate(nodes_dataset):
        if x[0] == y:
            coord[0]=i
        if x[1] == y:
            coord[1]=i
    target_edge_coord.append(coord)
    
# #convert the edge name to index using dictionary
# target_edge_coord = {}
# for x in target_edge:
#     target_edge_coord[x.tolist()] = ['nan', 'nan']
#     for i, y in enumerate(nodes_dataset):
#         if x[0] == y:
#             target_edge_coord[x]=i
#         if x[1] == y:
#             target_edge_coord[x]=i
    
#interaction values extraction using coordinates

interaction_values = []
for x in target_edge_coord:
    interaction_values.append(PPR_dataset[x[0],x[1]])

#scale interaction values to between 0 and 10
influence_values = influence_node(interaction_values)


# Then, for each target of a certain compound, we kept proteins with a non-0 influence score (the target itself was given a score of ten  and, when one protein was influenced by more than one target, the maximum score was kept). 
influenced_stringDB = {}
for x, y in zip(interaction_values, target_edge):
    if x == 0:
        continue
    if y[1] not in influenced_stringDB.keys():
        influenced_stringDB[y[1]] = x
    else:
        if influenced_stringDB[y[1]] < x:
            influenced_stringDB[y[1]] = x
        else:
            continue
    if y[0] not in influenced_stringDB.keys():
        influenced_stringDB[y[0]] = 10


adding weight from B4

In [35]:
in_threshold = pd.concat([kinase_ic_filter, g_protein_ic_filter,nuclear_receptor_ic_filter,ion_channel_ic_filter,other_ic_filter])

# Finally, these scores were multiplied by the weight of the compound-target annotation (see B4). 
#is everthing that with in class specific cut of multiplied by 1 and everything outside the class specific cutoff multiplied by log10
for k, v in influenced_stringDB.items():
    string_code = k.decode('ascii')
    uniprot_code = stringDB_uniprot_conv[stringDB_uniprot_conv['protein1']==string_code]['protein1_uniprot_id'].unique()[0]
    # uniprot = stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id']==uniprot_code]['protein1_uniprot_id'].unique()[0]
    if uniprot_code in in_threshold['uniprot_id'].unique():
        continue
    else:
        influenced_stringDB[k] = np.log10(v)

In [36]:
influenced_stringDB

{b'9606.ENSP00000216862': 0.015335382,
 b'9606.ENSP00000178638': 10,
 b'9606.ENSP00000300900': 0.0425477,
 b'9606.ENSP00000262418': -1.8118263,
 b'9606.ENSP00000285379': 10,
 b'9606.ENSP00000295256': 0.0023837313,
 b'9606.ENSP00000250559': -2.6582344,
 b'9606.ENSP00000382004': -2.7136254,
 b'9606.ENSP00000265686': -2.3814678,
 b'9606.ENSP00000261769': -2.916688,
 b'9606.ENSP00000393557': -1.6337378,
 b'9606.ENSP00000373620': 0.0017948113,
 b'9606.ENSP00000378920': -1.635768,
 b'9606.ENSP00000304669': -2.6967418,
 b'9606.ENSP00000366641': -1.5092093,
 b'9606.ENSP00000285381': 10,
 b'9606.ENSP00000358107': 0.1276431,
 b'9606.ENSP00000367608': 0.017717116,
 b'9606.ENSP00000405899': -1.5747824,
 b'9606.ENSP00000311165': 1.0,
 b'9606.ENSP00000264938': 0.009327489,
 b'9606.ENSP00000318355': -1.4979736,
 b'9606.ENSP00000220764': -1.9046055,
 b'9606.ENSP00000345659': 10,
 b'9606.ENSP00000300215': -1.707651,
 b'9606.ENSP00000430656': 10,
 b'9606.ENSP00000427690': -1.6068602,
 b'9606.ENSP0000037

#### Interaction3D

In [300]:
# an in-house network of physical binary PPIs69 (13,038 p, 64,659 i),  #!need to find in house pipeline
import csv
# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open("../interactions_interactome_3d.dat").readlines()]
interaction3D_df = pd.DataFrame(datContent, columns = datContent[0])
interaction3D_df = interaction3D_df[1:] 
interaction3D_df #! how did you get binary PPI interactome info from this?  https://interactome3d.irbbarcelona.org/help.php#interaction_page
# https://interactome3d.irbbarcelona.org/help.php#interaction_page

,PROT1,PROT2,RANK_MAJOR,RANK_MINOR,TYPE,PDB_ID,BIO_UNIT,CHAIN1,MODEL1,SEQ_IDENT1,...,SEQ_END1,DOMAIN1,CHAIN2,MODEL2,SEQ_IDENT2,COVERAGE2,SEQ_BEGIN2,SEQ_END2,DOMAIN2,FILENAME
1,A0A024RAV5,P02647,1,0,Structure,2mse,1,B,0,100.0,...,185,-,A,0,100.0,74.2,68,265,-,A0A024RAV5-P02647-EXP-2mse.pdb1-B-0-A-0.pdb
2,A0A024RAV5,P02647,2,0,Structure,6pts,1,B,0,100.0,...,185,-,A,0,100.0,74.2,68,265,-,A0A024RAV5-P02647-EXP-6pts.pdb1-B-0-A-0.pdb
3,A0A024RAV5,P10398,1,0,Structure,2mse,1,B,0,100.0,...,185,-,D,0,100.0,12.0,19,91,-,A0A024RAV5-P10398-EXP-2mse.pdb1-B-0-D-0.pdb
4,A0A075B5G3,Q8NBP7,1,0,Structure,4ov6,1,F,0,100.0,...,99,-,B,0,95.6,42.5,153,446,-,A0A075B5G3-Q8NBP7-EXP-4ov6.pdb1-F-0-B-0.pdb
5,A0A075B5G3,Q8NBP7,2,0,Structure,4ov6,2,G,0,100.0,...,96,-,E,0,92.2,42.3,153,445,-,A0A075B5G3-Q8NBP7-EXP-4ov6.pdb2-G-0-E-0.pdb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124919,V9HW83,V9HW83,54,0,Structure,4wb9,1,A,2,100.0,...,501,-,A,3,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-2-A-3.pdb
124920,V9HW83,V9HW83,55,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,1,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-1.pdb
124921,V9HW83,V9HW83,56,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,2,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-2.pdb
124922,V9HW83,V9HW83,57,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,3,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-3.pdb


In [145]:
bla = np.append(interaction3D_df['PROT1'].unique(), interaction3D_df['PROT2'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
bla = set(bla.tolist())

protein_encoding = {}
for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in interaction3D_df['PROT1']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in interaction3D_df['PROT2']:
    protein2_encoding.append(protein_encoding[x])
    
interaction3D_df['PROT1_encoding'] = protein1_encoding
interaction3D_df['PROT2_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('interaction3D_index.txt', sep = " ", index = False, index_label = False, header = False)
interaction3D_df[['PROT1_encoding','PROT2_encoding']].to_csv('interaction3D_edge.txt', sep = " ", index = False, index_label = False, header = False)

getting the uniprot correspondance for interactome3D

In [418]:
drugbank_df_nonan = drugbank_df[drugbank_df['pdb_list']!='Nan']


In [425]:
PROT_uniprot = {}
for index, row in tqdm(interaction3D_df.iterrows()):
    for i, x in drugbank_df_nonan.iterrows():
        if x['pdb_list'] == ['Nan']:
            continue
        # if bool(set(row['PDB_ID']) & set(x['pdb_list'].split(';'))):
        if row['PDB_ID'] in x['pdb_list'].split(';'):
            PROT_uniprot[row['PROT1']] = x['target_uniprot']

124923it [14:31:43,  2.39it/s]


In [427]:
import pickle
dict = {'Python' : '.py', 'C++' : '.cpp', 'Java' : '.java'}

# create a binary pickle file 
f = open("DB/PROT_uniprot.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(PROT_uniprot,f)

# close file
f.close()

infile = open("DB/PROT_uniprot.pkl",'rb')
PROT_uniprot = pickle.load(infile)
infile.close()

In [457]:
PROT1_uniprot = []
PROT2_uniprot = []

for x in interaction3D_df['PROT1']:
    try:
        PROT1_uniprot.append(PROT_uniprot[x])
    except:
        PROT1_uniprot.append('Nan')
        
for x in interaction3D_df['PROT2']:
    try:
        PROT2_uniprot.append(PROT_uniprot[x])
    except:
        PROT2_uniprot.append('Nan')
        
interaction3D_df['PROT1_uniprot'] = PROT1_uniprot
interaction3D_df['PROT2_uniprot'] = PROT2_uniprot
#drop all NA
interaction3D_df_drop = interaction3D_df[interaction3D_df['PROT1_uniprot']!='Nan']
interaction3D_df_drop = interaction3D_df_drop[interaction3D_df_drop['PROT2_uniprot']!='Nan']

In [479]:
# #produced uniprot matrix cannot be run
# bla = np.append(interaction3D_df_drop['PROT1_uniprot'].unique(), interaction3D_df_drop['PROT2_uniprot'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
# bla = set(bla.tolist())

# protein_encoding = {}
# for i, x in enumerate(bla):
#     protein_encoding[x] = i+1
# protein1_encoding = []
# for x in interaction3D_df_drop['PROT1_uniprot']:
#     protein1_encoding.append(protein_encoding[x])
    
# protein2_encoding = []
# for x in interaction3D_df_drop['PROT2_uniprot']:
#     protein2_encoding.append(protein_encoding[x])
    
# interaction3D_df_drop['PROT1_encoding'] = protein1_encoding
# interaction3D_df_drop['PROT2_encoding'] = protein2_encoding

# protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

# #export index file
# protein_encoding[['Encodings','Names']].to_csv('interaction3D_index_uniprot.txt', sep = " ", index = False, index_label = False, header = False)
# interaction3D_df_drop[['PROT1_encoding','PROT2_encoding']].to_csv('interaction3D_edge_uniprot.txt', sep = " ", index = False, index_label = False, header = False)

In [481]:
bla = np.append(interaction3D_df_drop['PROT1'].unique(), interaction3D_df_drop['PROT2'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
bla = set(bla.tolist())

protein_encoding = {}
for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in interaction3D_df_drop['PROT1']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in interaction3D_df_drop['PROT2']:
    protein2_encoding.append(protein_encoding[x])
    
interaction3D_df_drop['PROT1_encoding'] = protein1_encoding
interaction3D_df_drop['PROT2_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('interaction3D_index1.txt', sep = " ", index = False, index_label = False, header = False)
interaction3D_df_drop[['PROT1_encoding','PROT2_encoding']].to_csv('interaction3D_edge1.txt', sep = " ", index = False, index_label = False, header = False)

opening interactome3D h5

In [634]:
import h5py
with h5py.File('/Users/linfengwang/Github/linfeng-wang-CCdatabase/hotnet_output/interaction3D_mat1_ppr_0.6.h5', "r") as hdf:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % hdf.keys())
    PPR = hdf.get('PPR')
    PPR_dataset = np.array(PPR)
    
    edges = hdf.get('edges') #sample name, given in pairs
    edges_dataset = np.array(edges)

    nodes = hdf.get('nodes') #individual sample names
    nodes_dataset = np.array(nodes) 


Keys: <KeysViewHDF5 ['PPR', 'beta', 'edges', 'network_name', 'nodes']>


In [635]:
#get the edges
target_edge = []
for target in target_list_uniprot:
    for x in edges_dataset:
        # print(x[0].decode('ascii'))
        if PROT_uniprot[x[0].decode('ascii')] == target:
            target_edge.append(x)

# is is only protein1 influencing protein2 right? 

# convert the edge name to index using dictionary
target_edge_coord = {}
for x in target_edge:
    target_edge_coord[str(x)] = ['nan', 'nan']
    for i, y in enumerate(nodes_dataset):
        if x[0] == y:
            target_edge_coord[str(x)][0]=i
        if x[1] == y:
            target_edge_coord[str(x)][1]=i
    # target_edge_coord.append(coord)

#ALTERNATIVELY
# #convert the edge name to index using dictionary
# target_edge_coord = {}
# for x in target_edge:
#     target_edge_coord[x.tolist()] = ['nan', 'nan']
#     for i, y in enumerate(nodes_dataset):
#         if x[0] == y:
#             target_edge_coord[x]=i
#         if x[1] == y:
#             target_edge_coord[x]=i
    
#interaction values extraction using coordinates

interaction_values = []
for x in target_edge_coord.values():
    interaction_values.append(PPR_dataset[x[0],x[1]])

#scale interaction values to between 0 and 10
influence_values = influence_node(interaction_values)

In [636]:
# Then, for each target of a certain compound, we kept proteins with a non-0 influence score (the target itself was given a score of ten  and, when one protein was influenced by more than one target, the maximum score was kept). 
influenced_interactome3D = {}
for x, y in zip(interaction_values, target_edge):
    if x == 0:
        continue
    if y[1] not in influenced_interactome3D.keys():
        influenced_interactome3D[y[1]] = x
    else:
        if influenced_interactome3D[y[1]] < x:
            influenced_interactome3D[y[1]] = x
        else:
            continue
    if y[0] not in influenced_interactome3D.keys():
        influenced_interactome3D[y[0]] = 10


In [ ]:
# Finally, these scores were multiplied by the weight of the compound-target annotation (see B4). 
#is everthing that with in class specific cut of multiplied by 1 and everything outside the class specific cutoff multiplied by log10
for k, v in influenced_interactome3D.items():
    uniprot = stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id']==k.decode('ascii')]['protein1_uniprot_id'].unique()[0]
    if uniprot in in_threshold['uniprot_id'].unique():
        continue
    else:
        influenced_interactome3D[k] = np.log10(v)
        

In [637]:
influenced_interactome3D

{b'O60516': 0.13446733,
 b'O60573': 10,
 b'P61769': 0.008864713,
 b'P30511': 10,
 b'P04150': 0.02520138,
 b'O75376': 10,
 b'P10827': 0.06476477,
 b'P11473': 0.040337488,
 b'O15379': 0.25087938,
 b'P20393': 0.25087938,
 b'P10828': 0.039287344,
 b'Q07869': 0.041156508,
 b'O75469': 0.03423368,
 b'P55055': 0.031008163,
 b'P37231': 0.030559424,
 b'Q03181': 0.03292512,
 b'P13631': 0.045340285,
 b'P41182': 0.09091392,
 b'P63104': 0.009640468,
 b'Q96RR4': 10,
 b'P61981': 0.015860165,
 b'Q99748': 0.13376714,
 b'O00451': 10,
 b'P07949': 0.035068553,
 b'Q00526': 0.031774286,
 b'P24863': 10,
 b'Q13618': 0.049756836,
 b'O95198': 10,
 b'Q14145': 0.09303577,
 b'O94782': 0.06762086,
 b'P51784': 10,
 b'Q9Y2X8': 0.0076949713,
 b'Q8ND25': 10,
 b'P61077': 0.0059611006,
 b'P51965': 0.013047812,
 b'Q96LR5': 0.013631028,
 b'P0CG47': 0.005014507,
 b'Q16763': 10,
 b'P0CG48': 0.00410757,
 b'Q14494': 0.13460352,
 b'Q9Y5K5': 10}

#### Pathway Common

In [274]:
# a portion of Pathway Commons containing interactions from known pathways (Kyoto Encylopedia of Genes and Genomes65, NetPath66, PANTHER67 and WikiPathways68) (9,344 p, 242,962 i), #!need to find out
panther = pd.read_csv('../C5_PathwayCommons13.panther.hgnc.txt', sep='\t')
netpath = pd.read_csv('../C5_PathwayCommons13.netpath.hgnc.txt', sep = '\t')
wikipath = pd.read_csv('../C5_PathwayCommons.wiki.hgnc.txt', sep='\t')

netpath = netpath[netpath['INTERACTION_DATA_SOURCE']=='NetPath']
panther = panther[panther['INTERACTION_DATA_SOURCE']=='PANTHER']
pathway_common = pd.concat([panther, netpath])

pathway_common = pd.read_csv('../C5_PathwayCommons13.All.hgnc.txt', sep='\t')

pathway_common =pd.concat([pathway_common, wikipath])
pathway_common = pathway_common[pathway_common['INTERACTION_DATA_SOURCE'].isin(['NetPath','PANTHER','KEGG','WikiPathways'])]



In [276]:
pathway_common.head()

,PARTICIPANT_A,INTERACTION_TYPE,PARTICIPANT_B,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID,PATHWAY_NAMES,MEDIATOR_IDS
350,A4GALT,catalysis-precedes,ABO,KEGG,NaN,Glycosphingolipid biosynthesis - globo series;...,http://pathwaycommons.org/pc13/Catalysis_4f0b2...
351,A4GALT,catalysis-precedes,AK3,KEGG,NaN,Glycosphingolipid biosynthesis - globo series;...,http://pathwaycommons.org/pc13/BiochemicalReac...
352,A4GALT,catalysis-precedes,ALG13,KEGG,NaN,Glycosphingolipid biosynthesis - globo series;...,http://pathwaycommons.org/pc13/BiochemicalReac...
353,A4GALT,catalysis-precedes,ALG14,KEGG,NaN,Glycosphingolipid biosynthesis - globo series;...,http://pathwaycommons.org/pc13/BiochemicalReac...
354,A4GALT,catalysis-precedes,ALG5,KEGG,NaN,Glycosphingolipid biosynthesis - globo series;...,http://pathwaycommons.org/pc13/Catalysis_d5de5...


In [216]:
pathway = pd.read_csv('../pathwaycommon_uniprot.txt', sep='\t')


generating input file

In [220]:
# Opening JSON file
with open('../C5_pathway_common_uniprot.json') as f:
    # returns JSON object as 
    # a dictionary
    data = json.load(f)


In [333]:
Uniprot_pc = []
HGNC = []
for x in tqdm(data):
    try:
        x['HGNC Symbol']
    except:
        continue
    else:
        if len(x['HGNC Symbol']) == 0:
            continue
        if len(x['HGNC Symbol']) != len(x['UniProt']):
            continue
        else:
            Uniprot_pc.append(x['UniProt'])
            HGNC.append(x['HGNC Symbol'])
        

100%|██████████| 2281449/2281449 [00:02<00:00, 981983.83it/s] 


In [334]:
print(len(Uniprot_pc), len(HGNC))

2065533 2065533


In [335]:
#dont run this twice 
Uniprot_pc = [item for sublist in Uniprot_pc for item in sublist]
HGNC = [item for sublist in HGNC for item in sublist]


In [341]:
Uniprot_HGNC = pd.DataFrame(list(zip(Uniprot_pc, HGNC)), columns =['Uniprot_pc', 'HGNC'])
Uniprot_HGNC = Uniprot_HGNC.drop_duplicates()

In [381]:
#runs a long time
PARTICIPANT_A_uniprot = [0]*(pathway_common.shape[0])
PARTICIPANT_B_uniprot = [0]*(pathway_common.shape[0])

for i ,(x, y) in tqdm(enumerate(zip(pathway_common['PARTICIPANT_A'],pathway_common['PARTICIPANT_B']))):
    rowA = Uniprot_HGNC[Uniprot_HGNC["HGNC"]==x]
    rowB = Uniprot_HGNC[Uniprot_HGNC["HGNC"]==y]
    # print(i, rowA, rowB)
    if len(rowA['Uniprot_pc'].values) > 0:
        PARTICIPANT_A_uniprot[i]=rowA['Uniprot_pc'].values[0]
    else:
        PARTICIPANT_A_uniprot[i]=0
        
    if len(rowB['Uniprot_pc'].values) > 0:
        PARTICIPANT_B_uniprot[i]=rowB['Uniprot_pc'].values[0]
    else:
        PARTICIPANT_B_uniprot[i]=0 #missing uniprot id are labeled as zero

53825it [13:58, 64.18it/s]


In [395]:
# pathway_common.to_csv('DB/pathway_common_uniprot.csv')
pathway_common = pd.read_csv('DB/pathway_common_uniprot.csv')

In [399]:
#get rid of all the missing uniprot id that are labeled as zero
pathway_common = pathway_common[pathway_common['PARTICIPANT_A_uniprot']!=0]
pathway_common = pathway_common[pathway_common['PARTICIPANT_B_uniprot']!=0]

In [407]:
protein1_encoding

[]

In [409]:
#generating input for hotnet2 using uniprot value
bla = np.append(pathway_common['PARTICIPANT_A_uniprot'].unique(), pathway_common['PARTICIPANT_B_uniprot'].unique()) #this needs to be done as there are unique prot in PARTICIPANT_B that don't exist in PARTICIPANT_A
bla = set(bla.tolist())

protein_encoding = {}
for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in pathway_common['PARTICIPANT_A_uniprot']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in pathway_common['PARTICIPANT_B_uniprot']:
    protein2_encoding.append(protein_encoding[x])
    
pathway_common['PARTICIPANT_A_encoding'] = protein1_encoding
pathway_common['PARTICIPANT_B_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('pathway_common_uniprot_index.txt', sep = " ", index = False, index_label = False, header = False)
pathway_common[['PARTICIPANT_A_encoding','PARTICIPANT_B_encoding']].to_csv('pathway_common_uniprot_edge.txt', sep = " ", index = False, index_label = False, header = False)

Opening pathway common h5

In [640]:
import h5py
with h5py.File('/Users/linfengwang/Github/linfeng-wang-CCdatabase/hotnet_output/pathway_common_uniprot_mat_ppr_0.6.h5', "r") as hdf:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % hdf.keys())
    PPR = hdf.get('PPR')
    PPR_dataset = np.array(PPR)
    
    edges = hdf.get('edges') #sample name, given in pairs
    edges_dataset = np.array(edges)

    nodes = hdf.get('nodes') #individual sample names
    nodes_dataset = np.array(nodes) 


Keys: <KeysViewHDF5 ['PPR', 'beta', 'edges', 'network_name', 'nodes']>


In [643]:
#get the edges
target_edge = []
for target in target_list_uniprot:
    for x in edges_dataset:
        # print(x[0].decode('ascii'))
        if x[0].decode('ascii') == target:
            target_edge.append(x)

# convert the edge name to index using dictionary
target_edge_coord = {}
for x in target_edge:
    target_edge_coord[str(x)] = ['nan', 'nan']
    for i, y in enumerate(nodes_dataset):
        if x[0] == y:
            target_edge_coord[str(x)][0]=i
        if x[1] == y:
            target_edge_coord[str(x)][1]=i
    # target_edge_coord.append(coord)

interaction_values = []
for x in target_edge_coord.values():
    interaction_values.append(PPR_dataset[x[0],x[1]])

#scale interaction values to between 0 and 10
influence_values = influence_node(interaction_values)

# Then, for each target of a certain compound, we kept proteins with a non-0 influence score (the target itself was given a score of ten  and, when one protein was influenced by more than one target, the maximum score was kept). 
influenced_pathwaycommon = {}
for x, y in zip(interaction_values, target_edge):
    if x == 0:
        continue
    if y[1] not in influenced_pathwaycommon.keys():
        influenced_pathwaycommon[y[1]] = x
    else:
        if influenced_pathwaycommon[y[1]] < x:
            influenced_pathwaycommon[y[1]] = x
        else:
            continue
    if y[0] not in influenced_pathwaycommon.keys():
        influenced_pathwaycommon[y[0]] = 10

In [644]:
# Finally, these scores were multiplied by the weight of the compound-target annotation (see B4). 
#is everthing that with in class specific cut of multiplied by 1 and everything outside the class specific cutoff multiplied by log10
for k, v in influenced_pathwaycommon.items():
    uniprot = stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id']==k.decode('ascii')]['protein1_uniprot_id'].unique()[0]
    if uniprot in in_threshold['uniprot_id'].unique():
        continue
    else:
        influenced_pathwaycommon[k] = np.log10(v)
        

In [645]:
influenced_pathwaycommon

{b'Q9NZB8': -1.2082998,
 b'P00915': 10,
 b'P09622': -2.5057056,
 b'Q9Y5L3': 0.01193666,
 b'O60645': -1.1834769,
 b'P00918': 10,
 b'O60500': -2.5509589,
 b'O95267': -2.5228362,
 b'P41743': 0.0032394952,
 b'Q15139': 0.00536399}

In [646]:
influence_concat = {**influenced_stringDB, **influenced_interactome3D, **influenced_pathwaycommon}


#### Recon3D

In [648]:
# a network of metabolic genes based on Recon (v.2, http://vmh.uni.lu) (1,628 p, 246,937 i). #! recon https://www.vmh.life/#home matlab file?
import scipy.io
Recon3DModel_mat =  ('../Recon3D_301/Recon3DModel_301.mat')
Recon3D_mat = scipy.io.loadmat('../Recon3D_301/Recon3D_301.mat')

# To build this last network, we linked two metabolic proteins (enzymes or transporters) when the product metabolite of the first was the substrate of the second, 
# or when both were needed to perform a certain reaction, suggesting that they are part of the same protein complex. 
# Edges between proteins were weighted inversely proportional to the number of reactions involving their shared metabolites, 
# so that ‘currency’ metabolites such as ATP and water had marginal impact on the network connectivity. To control for indirect associations, 
# we deconvoluted the network using edge weights and setting a network deconvolution score cutoff of 0.9

In [649]:
Recon3D_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Recon3D'])

: 

In [7]:
with open('DB/recon_3d.txt', 'w') as f:
    writer = csv.writer(f)
    for x in range(1,len(Recon3D_mat['Recon3D'][0][0])):
        writer.writerow(Recon3D_mat['Recon3D'][0][0][x].tolist())


In [ ]:
Recon3D_mat['Recon3D'][0][0][6].tolist()

In [94]:
Recon3D_mat['Recon3D'][0][0][4]

array([[array(['10FTHF5GLUtl'], dtype='<U12')],
       [array(['10FTHF5GLUtm'], dtype='<U12')],
       [array(['10FTHF6GLUtl'], dtype='<U12')],
       ...,
       [array(['NADH2_u10mi'], dtype='<U11')],
       [array(['CYOOm3i'], dtype='<U7')],
       [array(['CYOOm2i'], dtype='<U7')]], dtype=object)

In [93]:
len(Recon3D_mat['Recon3D'][0][0][1])

8399

In [283]:
Recon3DModel_mat

NameError: name 'Recon3DModel_mat' is not defined

In [65]:
#network processing


Hot net

In [ ]:
# https://pypi.org/project/decorator/3.3.2/ 
# install decorator